<a href="https://colab.research.google.com/github/Shuichiro-T/gijutsushoten7_raspi_with_edgetpu_programs/blob/master/chap3/EdgeWithRaspi3v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tensorflowを1.14にダウングレードする。インストール後ランタイムの再起動を求められるので再起動する。

In [ ]:
!pip install tensorflow==1.14

ライブラリのインポートを行う。

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

MNISTデータのダウンロードを行い、画像データの正規化する（０～１の範囲にする）ために255で割る。

In [ ]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

学習モデルの生成を行う。

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

学習と評価を行う。

In [ ]:
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

モデルを保存するためにGoogle Driveのアクセス権を確認する。出力されたURLへアクセスし、認証を行った後に出力されるコードをコピーしダイアログへ入力する。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

学習したモデルをGoogle Driveへ出力する。

In [ ]:
model.save(
    '/content/drive/My Drive/Colab Notebooks/mymnist.h5')

出力したモデルを量子化しTensorFlow Liteのモデルへ変換する。

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(
    '/content/drive/My Drive/Colab Notebooks/mymnist.h5')
converter.inference_type = tf.lite.constants.QUANTIZED_UINT8
converter.default_ranges_stats = (0, 255)
input_arrays = converter.get_input_arrays()
converter.quantized_input_stats = {input_arrays[0] : (0., 1.)} 
tflite_model = converter.convert()

TensorFlow LiteのモデルをGoogle Driveへ保存する。

In [ ]:
open('/content/drive/My Drive/Colab Notebooks/mymnist.tflite', 
     "wb").write(tflite_model)


Edge TPUコンパイラをインストールする。

In [ ]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg \
 | sudo apt-key add -
!echo \
 "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" \
 | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get
!sudo apt-get install edgetpu-compiler

Edge TPU用に変換をする。

In [ ]:
!edgetpu_compiler \
 "/content/drive/My Drive/Colab Notebooks/mymnist.tflite" \
 -o "/content/drive/My Drive/Colab Notebooks/" \
 -m 10